In [2]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import ast
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import string
from IPython.display import Markdown
import textdistance
from fonctions import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tqdm.pandas()

In [2]:
texte_1 = 'sane cum de culminis nostri fulgore, velut ex sole radii, sic omnium largiente conditore prodeant dignitates ut prime lucis integritas minorati luminis non sensiat detrimenta, nam tanto magis regale sceptrum extollitur tantoque curia regiminis sollicitudinibus relevatur, quanto tribunal ipsius digniores circumspicit contribules ; hac igitur consideratione ducti, quia regni Francie moderamur habenas, solii nostri decus tam veterum dignitatum ornatibus confoveri quam etiam novis ardemus honoribus ampliari, et licet ad quorumlibet nostrorum fidelium munifice prosequenda servicia liberalitas nostra quadam generalitate sit proclivis, illis tamen est quadam specialitate debitrix et eorum vota prosequi liberaliter affectamus, in quibus sanguinis connexitatem et vinculum agnoscimus et puram fidei constanciam invenimus, ut ipsi, fervore devocionis accensi, reddantur de bono in melius circa majestatem regiam et ejus obsequia promptiores, et ne percepta dudum obsequia que conscribi perhenniter in libro memorie meruerunt preminentia gradus aboleat vel incompensata relinquat.'

pierre_1a = 'De fulgore throni cesarei, velut ex sole radii, sic certe prodeunt dignitates, ut prime lucis integritas non sentiat minorati luminis detrimenta, tantoque magis imperiale sceptrum extollitur, tanto cura regiminis pluria sollicitudinibus relevatur, quanto tribunal ipsius digniores in circuitu circumspicit contribules regiones. Hac igitur consideratione commoti, qui celesti providentia romani imperii moderamur habenas, solii nostri decus tam veterum dignitatum ornatibus confovemus, quam novis honoribus ampliamur.'

pierre_1b = 'Licet ad quorumlibet nostrorum fidelium, munifice prosequenda servicia liberalitatis nostre manus quadam generalitate sit habilis, illis tamen quadam specialitate fit debitrix, in quibus sanguinis idemptitatem agnoscimus et fidei pure constanciam invenimus […].'

In [3]:
texte_2 = 'decet excellentiam regiam pene duriciem quam juris rigor dictat aliquando pietatis mitigare, cum per hoc subditorum succrescat devocio et honor regius occulis augeatur.'

pierre_2 = 'satis nobis cedit ad gloriam et nostris fidelibus ad profectum, si motus nostros, quos armat potentia, sic mansuetudine decoremus, ut pene duriciem quam iuris rigor aliquando dictat in subditis clementie nostre molliamus unguentis, cum crescat in subditis devocionis vigor, dum pena remittitur, et dum devotorum succrescit cumulus nostri honoris per consequens titulus augeatur.'

In [4]:
texte_3 = 'Decus est principum subditorum votis clementer annuere ac eorum preces misericorditer exaudire ; etsi ad prosequenda vota fidelium regia liberalitas regulariter sit habilis, illud tamen occurrit precipuum, ut hiis qui reipublice se exposcunt, prefata liberalitas specialitate quadam graciosior et liberior porrigatur, ut sic per gratiam diligantur in populis et per Dominum tueantur in terris.'

pierre_3a = 'Decus est principum fidelium votis clementer annuere, ac supplicum preces misericorditer exaudire, ut sic per gratiam diligantur in populis et per dominium tueantur in terris.'

pierre_3b = 'licet ad prosequenda munificentie sic vota fidelium liberalitatis regie dextera generali quadam irregularitate sic sit habilis, illis tamen graciosa porrigitur quadam specialitate liberior, quos ad obsequia grata continuos fructus sic et utiles semper claris indiciis experimur.'

In [5]:
texte_4 = 'Dignum esse credimus ac consonum racioni atque concordabile reputamus ut hiis quos in legitimis actibus deffectus natalium impedit legitimacionis honor per principem reperiatur et, si quando super hiis magestatis regie favor seu benivolentia fidelium suorum aut eorum, qui regie magestatis fideles esse desiderant et in eadem fidelitate manere et perseverare noscuntur, supplicacione requiritur, liberaliter largiatur.'

pierre_4 = 'Dignum esse decrevimus et consentaneum rationi, ut hii quos interdum in legittimis actibus defectus natalium impedit, legittimacionis honore per principem reparentur, et si quando super hiis imperialis favor fidelium suorum supplicatione requiritur, liberaliter largiatur […].'


In [6]:
texte_5 = 'dignum esse credimus et summo Regi placere, si lapsis interdum per culpe obvicium celeri remedio subvenimus, cum per hoc crescat vigor in subditis et nostri honoris cumulus augeatur.'

pierre_5a = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus […].'

pierre_5b = 'cum crescat in subditis vigor, dum pena remittitur, et dum devotorum succrescit cumulus nostri honoris per consequens titulus augeatur.'

In [7]:
texte_6 = 'dignum opus prosequi credimus, innata nobis regali clemencia suadente, si interdum per lubrice culpe lapsis convicium ad indulgendum se nostra manus habilitet, rigorem tamen nostram taliter temperando quod partes justicie totaliter non excludat ; et licet ad hoc libenter erga omnes clemencie regie dexteram extendamus, illam tamen circa illos libentius extendimus in quibus devocionis et pure fidelitatis constanciam inviolabiliter reperimus.'

pierre_6a = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia, cito se nostra manus habilitat, et indulgemus obnoxiis, veniam post reatum, pulcrum vindicte genus exstimantes ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'

pierre_6b = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus. cum summum vindicte genus credentes ignoscere nolumus semper culpas delere per gladium, in quibus rigorem possumus temperamento clementie mitigare.'

pierre_6c = 'Quam sit favorabilis magnificentia cesaris et quam sit affabilis misericordiam petentibus vultus eius ex presentis negocii serie potestis aperte convincere, in quo sic est operatus iusticiam, ut per misericordie semitas dirigat gressus suos, sic misericorditer est profectus, ut partes iusticie non excludat […].'


In [8]:
texte_7 = 'egregium humanitatis opus in subditis agere credimus, innata nobis regali clementia suadente, si interdum per lubrica culpe delinimenta collapsis et veniam petentibus post reatum, humane condicionis fragilitate pensata, lenimento misericordie subvenimus, cum fervor devocionis et fidei sinceritas in subditis augeatur qui a culpis per regalem clementiam abluuntur.'

pierre_7a = 'nobile opus sedentis in solio clementia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia, cito se nostra manus habilitat et indulgemus obnoxiis veniam post reatum, pulcrum vindicte genus exstimantes ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'

pierre_7b = 'um crescat in eis etiam erga nos vigor devocionis et fidei, si fragilitatis humane memores aliquid severitati detrahimus, et que punire possemus in gladio, lenimento misericordie mitigamus.'

pierre_7c = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus, cum summum vindicte genus credentes ignoscere, nolumus semper culpas delere per gladium in quibus rigorem possumus temperamento clemencie mitigare […].'


In [9]:
texte_8 = 'egregium humanitatis opus in subditis prosequi credimus, innata nobis regali clementia suadente, si interdum per lubrice culpe lapsis convicium, humane condicionis fragilitate pensata, misericorditer subvenimus ; non est rigor justicie taliter in omnibus observandus, quin interdum possit et debeat linimento misericordie temperari.'

pierre_8 = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus, cum summum vindicte genus credentes ignoscere, nolumus semper culpas delere per gladium in quibus rigorem possumus temperamento clemencie mitigare […].'


In [10]:
texte_9 = 'lla decet regiam dignitatem frui mansuetudine et prerogativa virtutis ut, attendens quod, si culpa non precederet, locum venia non haberet, benignitas regia, si suum subditum quadam forsan necessitate potius quam voluntate a fide debita contigerit deviasse et, postmodum errorem suum sapienter advertens, ad gratiam reconciliationis admitti supplicat puro corde, paratis brachiis ipsum recipiat, oblita preterita culpa sua.'

pierre_9 = 'Illa fruimur ex imperii dignitate mansuetudine sic, ac prerogativa virtutis, ut quia nisi culpa precederet locum venia non haberet, sed quacumque hora subditus ad fidem nostram conversus fuerit, patulis brachiis recipiamus eumdem, obliti preterite culpe sue.'


In [11]:
texte_10 = 'In culpis delinquentium puniendis, personarum et temporum qualitate pensata, temperamus interdum gratie condimento rigorem.'

pierre_10 = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia cito se nostra manus habilitat, et indulgemus obnoxiis veniam post reatum, pulcrum vindicte genus exstimantes ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'


In [12]:
texte_11 = 'Inter cetera que sollicitudinis nostre cura quotidiana meditatione revolvit, illud occurrit nostre considerationi precipuum ut venerandus ortodoxe [sic] fidei nostre cultus nobis traditus per os prophetarum, apostolorum et quattuor evangelistarum nostre fidei catholice fidelium testium et scribarum, quorum sonus in orbem exiit universum, nostris temporibus augeatur subditique nostri in unionis federe pacis transquillitate gaudeant et fruantur ; ex hiis quidem potest augeri culminis nostri felicitas et subditorum nostrorum status servari pacificus, si Regi regum grato primitus devotionis obsequio placeamus et status ecclesiasticus in sua regularitate populusque nostro commissus regimini sub unionis nexu in sua integritate serventur.'

pierre_11 = 'Inter cetera, que sollicitudinis nostre cura cotidiana diligenti meditatione revolvit, illud occurrit nostre considerationi precipuum, qualiter venerandus orthodoxe fidei nostre cultus debito a nobis et nostris fidelibus affectu et devota veneratione colatur, ut quo pre ceteris principibus divina dextera nostrum erigit altius solium, eo pro [sic] per officiose gratitudinis debitum, nostra et nostrorum ubique devotio sibi solvat munera gratiarum. Ex eo si quidem potest regnantium felicitas augeri potissime et subditorum status servari pacificus, si Regi regum grato devotionis exennio placeamus, nec ex alicuius scandali suprectione dampnabili cultus debet postponi, vel intermitti dominicus, quia quo amplius humana scandala quandoque lasciviunt, eo sepefactus sedator esset devotius exorandus.'


In [13]:
texte_12 = 'Licet ad quorumlibet vota munifice prosequenda liberalitas nostra quadam generalitate sit proclivis, illis tamen est quadam specialitate debitrix quos ad regie celsitudinis grata obsequia promptos et utiles claris judiciis experimur.'

pierre_12a = 'Licet ad quorumlibet nostrorum fidelium, munifice prosequenda servicia liberalitatis nostre manus quadam generalitate sit habilis, illis tamen quadam specialitate fit debitrix, in quibus sanguinis idemptitatem agnoscimus et fidei pure constanciam invenimus […].'

pierre_12b = 'Licet ad prosequenda munificentie [sic] vota fidelium liberalitatis regie dextera generalita quadam irregularitate [sic] sit habilis, illis tamen graciosa porrigitur, quadam specialitate liberior, quos ad obsequia grata continuos fructus [sic] et utiles claris semper indiciis experimur.'


In [14]:
texte_13 = 'Opus insigne presidentis in solio in nobis clementia suggerente proficimus, si per lubrica culpe delinimenta labentibus misericordie celeri levamine subvenimus.'

pierre_13a = 'Opus in nobis egregium, clemencia suggerente, perficimus, si per lubrica culpe deliramenta labentibus, misericordie celeris remediosubvenimus '

pierre_13b = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus'


In [15]:
texte_14 = 'Pulcrum vindicte genus estimantes ignoscere, temperamus interdum gratie condimento rigorem.'

pierre_14 = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia, se cito nostra manus habilitat, et indulgemus obnoxiis veniam post reatum, pulcrum vindicte genus exstimantes, ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'


In [16]:
texte_15 = 'Satis nobis, quem Rex regum ad regie dignitatis fastigia provexit, cedit ad gloriam, si motus nostros virtute clementie perficimus et mansuetudinem in subditis excercemus, cum crescat etiam erga nos vigor devotionis et fidei, si, fragilitatis humane memores, aliquid severitati detrahimus et que punire possemus lenimento misericordie mittigamus.'

pierre_15 = 'Satis nobis cedit ad gloriam et regie dignitatis fastigia sublimamus sic, si motus nostros clemencie virtute perspicimus sic, et mansuetudinem in subditis exercemus, cum crescat in eis etiam erga nos vigor devocionis et fidei si fragilitatis humane memores aliquid severitati detrahimus, et que punire possemus in gladio, lenimento misericordie mitigamus'

In [17]:
characters_to_remove = string.punctuation + '[]' +'–'

In [18]:
x = remove_characters(texte_6,characters_to_remove).lower().lstrip().rstrip().replace('j','i')
y = remove_characters(pierre_6c,characters_to_remove).lower().lstrip().rstrip().replace('j','i')

In [19]:
un, deux = bold_texts(x,y)

In [20]:
liste_texte = [texte_1,texte_2,texte_3,texte_4,texte_5,texte_6,texte_7,texte_8,texte_9,texte_10,texte_11,
              texte_12,texte_13,texte_14,texte_15]
liste_pierre = [pierre_1a,pierre_1b,pierre_2,pierre_3a,pierre_3b,pierre_4,pierre_5a,pierre_5b,pierre_6a,
                pierre_6b,pierre_6c,pierre_7a,pierre_7b,pierre_7c,pierre_8,pierre_9,pierre_10,pierre_11,
               pierre_12a,pierre_12b,pierre_13a,pierre_13b,pierre_14,pierre_15]

indice_pierre = ['pierre_1a', 'pierre_1b', 'pierre_2', 'pierre_3a', 'pierre_3b', 'pierre_4',
                'pierre_5a', 'pierre_5b', 'pierre_6a', 'pierre_6b', 'pierre_6c', 'pierre_7a',
                'pierre_7b', 'pierre_7c', 'pierre_8', 'pierre_9', 'pierre_10', 'pierre_11',
                'pierre_12a', 'pierre_12b', 'pierre_13a', 'pierre_13b', 'pierre_14', 'pierre_15']

indice_texte = ['texte_1', 'texte_2', 'texte_3', 'texte_4', 'texte_5', 'texte_6', 'texte_7', 'texte_8', 
               'texte_9', 'texte_10', 'texte_11', 'texte_12', 'texte_13', 'texte_14', 'texte_15']


In [21]:
liste_texte = [process_string(remove_characters(texte, characters_to_remove)) for texte in liste_texte]
liste_pierre = [process_string(remove_characters(texte, characters_to_remove)) for texte in liste_pierre]

In [22]:
text_list = []
for item in liste_texte:
    text_list.extend([item]*len(liste_pierre))
pierre_list = liste_pierre * len(liste_texte)

text_index = []
for item in indice_texte:
    text_index.extend([item]*len(indice_pierre))
pierre_index = indice_pierre * len(indice_texte)

In [23]:
data = pd.DataFrame({'textes':text_list,'indice_text':text_index,'pierre':pierre_list,'indice_pierre':pierre_index})

In [24]:
data

,textes,indice_text,pierre,indice_pierre
0,sane cum de culminis nostri fulgore uelut ex s...,texte_1,de fulgore throni cesarei uelut ex sole radii ...,pierre_1a
1,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad quorumlibet nostrorum fidelium munifi...,pierre_1b
2,sane cum de culminis nostri fulgore uelut ex s...,texte_1,satis nobis cedit ad gloriam et nostris fideli...,pierre_2
3,sane cum de culminis nostri fulgore uelut ex s...,texte_1,decus est principum fidelium uotis clementer a...,pierre_3a
4,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad prosequenda munificentie sic uota fid...,pierre_3b
...,...,...,...,...
355,satis nobis quem rex regum ad regie dignitatis...,texte_15,licet ad prosequenda munificentie sic uota fid...,pierre_12b
356,satis nobis quem rex regum ad regie dignitatis...,texte_15,opus in nobis egregium clemencia suggerente pe...,pierre_13a
357,satis nobis quem rex regum ad regie dignitatis...,texte_15,nobile opus sedentis in solio clemencia suaden...,pierre_13b
358,satis nobis quem rex regum ad regie dignitatis...,texte_15,nobile opus sedentis in solio clemencia suaden...,pierre_14


In [25]:
x = liste_texte

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


textes = []
for i in tqdm(range(len(x))):
    sentences: List[str] = [x[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        textes.append(dic)
        
y = liste_pierre

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


pierre = []
for i in tqdm(range(len(y))):
    sentences: List[str] = [y[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        pierre.append(dic)

fatal: HEAD n'est pas un nom d'objet valide
100%|███████████████████████████████████████████| 24/24 [00:05<00:00,  4.10it/s]


In [26]:
pierre_lemmes = []   

for dic in range(len(pierre)):
    
    inter = []

    for i in pierre[dic]:
        inter.append(i.get('lemma'))
    pierre_lemmes.append(inter)
texte_lemmes = []   

for dic in range(len(textes)):
    
    inter = []

    for i in textes[dic]:
        inter.append(i.get('lemma'))
    texte_lemmes.append(inter)
    
    
pierre_pos = []   

for dic in range(len(pierre)):
    
    inter = []

    for i in pierre[dic]:
        inter.append(i.get('pos'))
    pierre_pos.append(inter)
texte_pos = []   

for dic in range(len(textes)):
    
    inter = []

    for i in textes[dic]:
        inter.append(i.get('pos'))
    texte_pos.append(inter)

In [27]:
#création du dictionnaire de pos

test = [item for sublist in pierre for item in sublist]
test2 = [item for sublist in textes for item in sublist]

test.extend(test2)
words = []
pos = []
lemmas = []

for dic in test:
    lemmas.append(dic.get('lemma'))
    pos.append(dic.get('pos'))
    words.append(dic.get('form'))
    

In [28]:
dico = pd.DataFrame({'forme':words,'lemmes':lemmas,'pos':pos})
unique = dico['forme'].drop_duplicates().to_frame()
unique = unique.reset_index()
indices = unique['index'].tolist()

In [29]:
dico = dico.loc[indices]
dico.to_excel('dico_test.xlsx')

In [30]:
# liste de mots sans stopwords pour les distances cosine
liste_pos = ['ADJqua','VER','ADV','NOMcom']
hadri = []
for i in range(len(pierre)):
    stratos = []
    for j in pierre[i]:
        if j.get('pos') in liste_pos:
            stratos.append(j.get('lemma'))
    hadri.append(stratos)

victor = []
for i in range(len(textes)):
    amelle = []
    for j in textes[i]:
        if j.get('pos') in liste_pos:
            amelle.append(j.get('lemma'))
    victor.append(amelle)
len(victor)

15

In [31]:
texte_lemme = []
for item in texte_lemmes:
    texte_lemme.extend([item]*len(pierre_lemmes))
pierre_lemme = pierre_lemmes * len(texte_lemmes)

pos_texte = []
for item in texte_pos:
    pos_texte.extend([item]*len(pierre_pos))
pos_pierre = pierre_pos * len(texte_pos)

'''texte_stop = []
for item in victor:
    texte_stop.extend([item]*len(hadri))
pierre_stop = hadri * len(victor)
'''
data['texte_lemmes'] = texte_lemme
data['pierre_lemmes'] = pierre_lemme
data['pierre_pos'] = pos_pierre
data['texte_pos'] = pos_texte
#data['texte_stop'] = texte_stop
#data['pierre_stop'] = pierre_stop

In [31]:
data.to_excel('pierre_textes.xlsx')

In [32]:
data['bold'] = data.progress_apply(lambda x: bold_texts(x['textes'],x['pierre']),axis=1)
data = data[data['bold'] != False].reset_index(drop=True)

100%|█████████████████████████████████████████| 360/360 [01:02<00:00,  5.73it/s]


['sane',
 'cum',
 'de',
 'culminis',
 'nostri',
 'fulgore',
 '**uelut**',
 '**ex**',
 '**sole**',
 '**radii**',
 '**sic**',
 'omnium',
 'largiente',
 'conditore',
 '******prodeant******',
 '******dignitates******',
 '**ut**',
 '****prime****',
 '**lucis**',
 '**integritas**',
 '**minorati**',
 '**luminis**',
 '**non**',
 '********sensiat********',
 '**detrimenta**',
 'nam',
 'tanto',
 '**magis**',
 'regale',
 '****sceptrum****',
 '****extollitur****',
 'tantoque',
 '********curia********',
 '****regiminis****',
 '****sollicitudinibus****',
 '**releuatur**',
 '**quanto**',
 '**tribunal**',
 '**ipsius**',
 '**digniores**',
 '**circumspicit**',
 '**contribules**',
 '**hac**',
 '**igitur**',
 '**consideratione**',
 'ducti',
 'quia',
 'regni',
 'francie',
 '**moderamur**',
 '**habenas**',
 '**solii**',
 '**nostri**',
 '**decus**',
 '**tam**',
 '**ueterum**',
 '**dignitatum**',
 '**ornatibus**',
 'confoueri',
 '**quam**',
 'etiam',
 'nouis',
 'ardemus',
 'honoribus',
 'ampliari',
 'et',
 'li

In [129]:
data['pierre_stop'] = data['pierre_stop'].apply(join)
data['texte_stop'] = data['texte_stop'].apply(join)
data['cosine'] = data.apply(lambda x:cosine_sim(x['texte_stop'],x['pierre_stop']),axis=1)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
data = data[['textes','indice_text','pierre','indice_pierre','texte_lemmes','pierre_lemmes','texte_pos','pierre_pos']]
data['quadri_commun'] = data.progress_apply(lambda x: avoir_quadrigramme_commun(x['textes'],x['pierre']),axis=1)

In [34]:
data['texte_bold'], data['pierre_bold'] = zip(*data['bold'].apply(from_tuples_to_column))

In [35]:
data = data[['textes','indice_text','pierre','indice_pierre','texte_bold','pierre_bold','texte_lemmes','pierre_lemmes','texte_pos','pierre_pos']]

In [36]:
data['etoiles_texte'] = data['texte_bold'].apply(indice_etoiles)
data['etoiles_pierre'] = data['pierre_bold'].apply(indice_etoiles)

In [37]:
data['indice_que_texte'] = data['texte_bold'].apply(index_que)
data['indice_que_pierre'] = data['pierre_bold'].apply(index_que)

In [38]:
data['lemmes_texte'] = data.apply(lambda x: jarod(x['texte_bold'],x['texte_lemmes']),axis=1).apply(transform_word)
data['lemmes_pierre'] = data.apply(lambda x:jarod(x['pierre_bold'],x['pierre_lemmes']),axis=1).apply(transform_word)

In [36]:
#data['compare'] = data.apply(lambda x:lengths(x['lemmes_pierre'],x['pierre_bold']),axis=1)

In [39]:
data['test_lemmes_texte'] = data.apply(lambda x: to_bold(x['lemmes_texte'],x['etoiles_texte']),axis=1).apply(transform_word)
data['test_lemmes_pierre'] = data.apply(lambda x: to_bold(x['lemmes_pierre'],x['etoiles_pierre']),axis=1).apply(transform_word)

In [40]:
def lemmas_bold_2(x,y):
    
    match = []
    liste_pos = ['ADJqua','VER','NOMcom','ADV']

    for i in range(len(x)):
        if not x[i].startswith('**') and x[i] in y:
            match.append(x[i])
    final = []

    for i in match:
        if new_dic.get(i) in liste_pos:
            final.append(i)

    for i in range(len(x)):
        if x[i] in final:
            x[i] = f'**{x[i]}**'

    for i in range(len(y)):
        if y[i] in final:
            y[i] = f'**{y[i]}**'
    
    return x,y

In [41]:
data['segment_texte'] = data.apply(lambda row: segment_lists(row['texte_bold'],row['etoiles_texte'][0],row['etoiles_texte'][-1],context_size=20), axis =1)
data['segment_pierre'] = data.apply(lambda row: segment_lists(row['pierre_bold'],row['etoiles_pierre'][0],row['etoiles_pierre'][-1],context_size=20), axis =1)

data['segment_texte_lemme'] = data.apply(lambda row: segment_lists(row['test_lemmes_texte'],row['etoiles_texte'][0],row['etoiles_texte'][-1],context_size=20), axis =1)
data['segment_pierre_lemme'] = data.apply(lambda row: segment_lists(row['test_lemmes_pierre'],row['etoiles_pierre'][0],row['etoiles_pierre'][-1],context_size=20), axis =1)

data['final'] = data.apply(lambda row: lemmas_bold_2(row['segment_texte_lemme'],row['segment_pierre_lemme']),axis=1)

data['texte_final_lemmes'], data['pierre_final_lemmes'] = zip(*data['final'].apply(from_tuples_to_column))
data['indice_final_texte'] = data['texte_final_lemmes'].apply(indice_etoiles)
data['indice_final_pierre'] = data['pierre_final_lemmes'].apply(indice_etoiles)

In [42]:
data['texte_final'] = data.apply(lambda x: to_bold(x['segment_texte'],x['indice_final_texte']),axis=1).apply(transform_word)
data['pierre_final'] = data.apply(lambda x: to_bold(x['segment_pierre'],x['indice_final_pierre']),axis=1).apply(transform_word)

In [45]:
data[data['indice_text'] == 'texte_1']

,textes,indice_text,pierre,indice_pierre,texte_bold,pierre_bold,texte_lemmes,pierre_lemmes,texte_pos,pierre_pos,...,segment_pierre,segment_texte_lemme,segment_pierre_lemme,final,texte_final_lemmes,pierre_final_lemmes,indice_final_texte,indice_final_pierre,texte_final,pierre_final
0,sane cum de culminis nostri fulgore uelut ex s...,texte_1,de fulgore throni cesarei uelut ex sole radii ...,pierre_1a,"[sane, cum, de, culminis, nostri, fulgore, **u...","[de, fulgore, throni, cesarei, **uelut**, **ex...","[sane, cum3, de, culmen, noster, fulgor, uelut...","[de, fulgor, thronus, cesareus, uelut1, ex, so...","[ADV, CONsub, PRE, NOMcom, PROpos, NOMcom, ADV...","[PRE, NOMcom, NOMcom, ADJqua, ADVrel, PRE, NOM...",...,"[de, fulgore, throni, cesarei, ****uelut****, ...","[sane, cum3, de, culmen, noster, fulgor, **uel...","[de, fulgor, thronus, cesareus, **uelut1**, **...","([sane, cum3, de, culmen, noster, fulgor, **ue...","[sane, cum3, de, culmen, noster, fulgor, **uel...","[de, fulgor, thronus, cesareus, **uelut1**, **...","[6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19, 20, 2...","[4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17...","[sane, cum, de, culminis, nostri, fulgore, **u...","[de, fulgore, throni, cesarei, **uelut**, **ex..."
1,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad quorumlibet nostrorum fidelium munifi...,pierre_1b,"[sane, cum, de, culminis, nostri, fulgore, uel...","[**licet**, **ad**, **quorumlibet**, **nostror...","[sane, cum3, de, culmen, noster, fulgor, uelut...","[licet3, ad2, quisque2, noster, fidelis2, muni...","[ADV, CONsub, PRE, NOMcom, PROpos, NOMcom, ADV...","[CONsub, PRE, PROind, PROpos, NOMcom, ADV, VER...",...,"[****licet****, ****ad****, ****quorumlibet***...","[quia, regnum, francies, moderor, habena, soli...","[**licet3**, **ad2**, **quisque2**, **noster**...","([quia, regnum, francies, moderor, habena, sol...","[quia, regnum, francies, moderor, habena, soli...","[**licet3**, **ad2**, **quisque2**, **noster**...","[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15,...","[quia, regni, francie, moderamur, habenas, sol...","[**licet**, **ad**, **quorumlibet**, **nostror..."
2,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad prosequenda munificentie sic uota fid...,pierre_3b,"[sane, cum, de, culminis, nostri, fulgore, uel...","[licet, ad, prosequenda, munificentie, sic, uo...","[sane, cum3, de, culmen, noster, fulgor, uelut...","[licet3, ad2, prosequor, munificentiens, sic, ...","[ADV, CONsub, PRE, NOMcom, PROpos, NOMcom, ADV...","[CONsub, PRE, VER, ADJqua, ADV, NOMcom, NOMcom...",...,"[licet, ad, prosequenda, munificentie, sic, uo...","[confoueo, quam1, etiam, nouus, ardeo, honor, ...","[licet3, ad2, prosequor, munificentiens, sic, ...","([confoueo, quam1, etiam, nouus, ardeo, honor,...","[confoueo, quam1, etiam, nouus, ardeo, honor, ...","[licet3, ad2, prosequor, munificentiens, sic, ...","[16, 20, 22, 23]","[7, 14, 16, 17]","[confoueri, quam, etiam, nouis, ardemus, honor...","[licet, ad, prosequenda, munificentie, sic, uo..."
3,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad quorumlibet nostrorum fidelium munifi...,pierre_12a,"[sane, cum, de, culminis, nostri, fulgore, uel...","[**licet**, **ad**, **quorumlibet**, **nostror...","[sane, cum3, de, culmen, noster, fulgor, uelut...","[licet3, ad2, quisque2, noster, fidelis2, muni...","[ADV, CONsub, PRE, NOMcom, PROpos, NOMcom, ADV...","[CONsub, PRE, PROind, PROpos, NOMcom, ADV, VER...",...,"[****licet****, ****ad****, ****quorumlibet***...","[quia, regnum, francies, moderor, habena, soli...","[**licet3**, **ad2**, **quisque2**, **noster**...","([quia, regnum, francies, moderor, habena, sol...","[quia, regnum, francies, moderor, habena, soli...","[**licet3**, **ad2**, **quisque2**, **noster**...","[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15,...","[quia, regni, francie, moderamur, habenas, sol...","[**licet**, 

In [54]:
x = data['texte_final'][0]
y = data['pierre_final'][0]

In [55]:
final_texte = ' '.join(x)
pierre_final = ' '.join(y)

print('----------')
display(Markdown(final_texte))
print('----------')
display(Markdown(pierre_final))    
print('----------')

----------


sane cum de culminis nostri fulgore **uelut** **ex** **sole** **radii** **sic** omnium largiente conditore **prodeant** **dignitates** **ut** **prime** **lucis** **integritas** **minorati** **luminis** **non** **sensiat** **detrimenta** nam **tanto** **magis** regale **sceptrum** **extollitur** tantoque **curia** **regiminis** **sollicitudinibus** **releuatur** **quanto** **tribunal** **ipsius** **digniores** **circumspicit** **contribules** **hac** **igitur** **consideratione** ducti quia regni francie **moderamur** **habenas** **solii** **nostri** **decus** **tam** **ueterum** **dignitatum** **ornatibus** confoueri **quam** etiam nouis ardemus **honoribus** ampliari et licet ad quorumlibet nostrorum fidelium munifice prosequenda seruicia liberalitas nostra quadam generalitate sit procliuis

----------


de fulgore throni cesarei **uelut** **ex** **sole** **radii** **sic** certe **prodeunt** **dignitates** **ut** **prime** **lucis** **integritas** **non** **sentiat** **minorati** **luminis** **detrimenta** tantoque **magis** imperiale **sceptrum** **extollitur** **tanto** **cura** **regiminis** pluria **sollicitudinibus** **releuatur** **quanto** **tribunal** **ipsius** **digniores** in circuitu **circumspicit** **contribules** regiones **hac** **igitur** **consideratione** commoti qui celesti prouidentia romani imperii **moderamur** **habenas** **solii** **nostri** **decus** **tam** **ueterum** **dignitatum** **ornatibus** confouemus **quam** nouis **honoribus** ampliamur

----------


# Test Viard

In [2]:
delete_quadrigramme = [['notum','facimus','uniuersis','tam'],['tam', 'presentibus', 'quam', 'futuris'],
                 ['uniuersis', 'tam', 'presentibus', 'quam'],['rex', 'notum', 'facimus', 'uniuersis'],
                 ['facimus', 'uniuersis', 'tam', 'presentibus'],['presentibus', 'quam', 'futuris', 'nos'],
                 ['presentibus', 'quam', 'futuris', 'quod'],['etc', 'notum', 'facimus', 'uniuersis'],
                 ['et', 'ex', 'certa', 'scientia'],['notum', 'facimus', 'uniuersis', 'presentibus'],
                 ['quam', 'futuris', 'quod', 'cum'],['quam', 'futuris', 'quod', 'nos'],
                 ['de', 'speciali', 'gratia', 'et'],['in', 'regno', 'nostro', 'usque'],
                 ['gratia', 'et', 'ex', 'certa'],['in', 'regno', 'nostro', 'predicto'],
                 ['presentibus', 'et', 'futuris', 'quod'],['notum', 'facimus', 'quod', 'cum'],
                 ['quam', 'futuris', 'nos', 'in'],['de', 'regno', 'nostro', 'in'],
                 ['et', 'ex', 'certa', 'sciencia'],['et', 'in', 'regno', 'nostro'],
                 ['in', 'regno', 'nostro', 'et'],['facimus', 'tam', 'presentibus', 'quam']]

In [3]:
file = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/PDV_grams.xlsx'
viard = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/Viard_grams.xlsx'

def delete_items_with_prefix(dictionnaire, prefix):
    keys_to_delete = [key for key in dictionnaire if key.startswith(prefix)]
    for key in keys_to_delete:
        del dictionnaire[key]


def is_single_word(s):
    return isinstance(s, str) and len(s.split()) == 1

def keys_with_single_word_value(dictionary):
    single_word_keys = []
    for key, value in dictionary.items():
        if is_single_word(value):
            single_word_keys.append(key)
    return single_word_keys
def join(liste):
    return ' '.join(liste)

df_pdv = pd.read_excel(file)
df_viard = pd.read_excel(viard)

df = pd.DataFrame({'texts':df_pdv['textes'],'clean_PDV': df_pdv['textes'].apply(clean_text)})
df['compare'] = df['clean_PDV'].apply(join)

df2 = pd.DataFrame({'viard_texts':df_viard['text'],'clean_viard':df_viard['text'].apply(clean_text)})

column_without_brackets = [[string.replace('[', '').replace(']', '').lstrip().rstrip() for string in sublist] for sublist in df2['clean_viard']]

df2['clean'] = column_without_brackets
df2['compare'] = df2['clean'].apply(join)
viard = {}
for i in range(len(df2['compare'].tolist())):
    viard[f'viard_{i+1}'] = df2['compare'].tolist()[i]
dico_un = {}

for i in range(len(df['compare'].tolist())):
    dico_un[f'pdv_{i+1}'] = df['compare'].tolist()[i]

pierre_de_la_vigne = list(dico_un.values())*len(viard)
cles_de_la_vigne = list(dico_un.keys())*len(viard)

jean_viard = copies(list(viard.values()),list(dico_un.values()))
cles_jean = copies(list(viard.keys()),list(dico_un.values()))

tqdm.pandas()
zoe = pd.DataFrame({'viard':jean_viard,'pdv':pierre_de_la_vigne})
viard_lemmes = list(viard.values())
viard_keys = list(viard.keys())
pdv_keys = list(dico_un.keys())
trois = copies(viard_keys,list(dico_un.values()))
quatre = pdv_keys*len(viard_keys)

zoe['indice_viard'] = trois
zoe['indice_pdv'] = quatre
zoe = zoe[['indice_viard','viard','indice_pdv','pdv']]




#zoe = zoe[['indice_viard','viard','indice_pdv','pdv','lemmes_viard','lemmes_pdv','cosine']]
#zoe = zoe[zoe['cosine'] > 0.05]


#zoe['quadri_viard'] = zoe['viard'].progress_apply(split).progress_apply(string_to_quadrigrams)
#zoe['quadri_pdv'] = zoe['pdv'].progress_apply(split).progress_apply(string_to_quadrigrams)
#zoe['compare'] = zoe.progress_apply(lambda x: list_of_common_quadrigrams(x['quadri_viard'],x['quadri_pdv']),axis=1)
'''oscar = zoe[zoe['compare'] != False].reset_index(drop=True)

oscar['compare_first'] = oscar['compare'].apply(keep_first_element)
oscar['delete'] = oscar.progress_apply(lambda x: delete_quadri(x['compare_first'],delete_quadrigramme),axis=1)
loic = oscar[oscar['delete'] != False].reset_index(drop=True)
loic['len'] = loic['delete'].apply(length)
loic = loic[loic['len'] > 1].reset_index(drop=True)
loic['same'] = loic['delete'].apply(same)
mathieu = loic[loic['same'] == False].reset_index(drop=True)'''

"oscar = zoe[zoe['compare'] != False].reset_index(drop=True)\n\noscar['compare_first'] = oscar['compare'].apply(keep_first_element)\noscar['delete'] = oscar.progress_apply(lambda x: delete_quadri(x['compare_first'],delete_quadrigramme),axis=1)\nloic = oscar[oscar['delete'] != False].reset_index(drop=True)\nloic['len'] = loic['delete'].apply(length)\nloic = loic[loic['len'] > 1].reset_index(drop=True)\nloic['same'] = loic['delete'].apply(same)\nmathieu = loic[loic['same'] == False].reset_index(drop=True)"

In [15]:
viard.get('viard_41')

'philippus dei etc notum facimus etc quod cum religiosus uir dilectus et fidelis petrus abbas monasterii cluniacensis  consiliarius noster supplicauerit nobis ut emptioni et acquisitioni per eum factis de duobus maneriis seu hospiciis cum suis iuribus et pertinenciis que titulo emptionis pro se suisque successoribus parisius acquisiuit uidelicet hospicium seu manerium situatum apud palatium termarum  quod emit philippo de tria  milite et hospicium seu manerium quod fuit gerardi de castellione militis domini de ruppe de milay quod iohannes de castrouillani et eius uxor uendiderunt eidem abbati que quidem hospicia seu maneria confrontantur seu continguantur religiosis sancti maturini etuallisde sernay ac uico per quem itur de sancto maturino predicto ad uicum cithare  ante domum domini de haricuria ex una parte et hospicio guillelmi de nangis alias dicti de palatio termarum et hospitiis uicecomitis de bello monte et de bersillo ac uico cythare ex altéra in censiua ecclesie sancte genouef

In [40]:
y = dico_un.get('pdv_198')

In [31]:
lettre = 'In nomine Domini nostri Ihesu Christi amen. Sollicitudinis et officii interesse dinoscitur pastoralis, ut summa animi vigilancia et consideracione perspicaci, illa que commisse sibi ecclesie profutura fuerint, procurare debeat mediatoris Dei et hominum vicarius et minister, et circumspecte provisionis studio promovere, quidquid in emolumentum ipsius commodum cedere viderit vel honorem, et tanto sollicicius curam ipsius agere, quanto, divinis affatibus contestatus, gregi sibi credito providere et honera pascendi gregis suis cum fratribus partiri cogitur ex precepto. Idcirco nos Andreas, Dei gracia episcopus ecclesie Poznaniensis, instanciis et crebrius venerabilis patris domini Iacobi, divina providencia sancte Gneznensis ecclesie archiepiscopi, hortati monitis et rogati: cum ecclesie nostre Poznaniensis archidiaconatus, canonice nunc vacans, ex distancia longiore et multiplici amplificatione et promixcione eciam et excrescencia creverit populorum, commodeque tribus archidiaconis in solidum regendus sufficiat et decenter, ipsum in tres archidiaconatus separatos canonice et disiunctos auctoritate nobis tradita divideremus, certis et determinatis limitibus declaratos: prefatis igitur venerabilis patris sanis exhortacionibus, salubribus consiliis et canonicis monitis inclinati, pocius animabus, de quibus in districto examine coram exactissimo iudice sumus racionem in die iudicii reddituri, et profectui ecclesie nostre intendere et consulere cupientes, diligenti et unanimi deliberacione prehabitis et tractatu, de consensu et beneplacito fratrum nostrorum Capituli Poznaniensis dictum archidiaconatum nostrum Poznaniensem in tres archidiaconatus dividimus sub hoc modo, videlicet quod hec ecclesie sue ex illa parte fluvii qui dicitur Varta, ad maiorem archidiaconatum nostre ecclesie sint spectantes: Mossina, Locza, Modrze, Granovo, Droszim, Slup, Neproszevo, Kasimirs, Szamantuli, Obriczko, et alie omnes versus Poznaniam site ab istis ecclesiis nominatis. Ab alia vero parte eiusdem fluvii, omnes ecclesie incipiendo a Censzim usque in Dirczen, ita tamen, quod se ultra Notes fluvium non extendat, quia illam partem archidiaconatui quarto reservamus. Medium vero archidiaconatum, sicut iste distingunt ecclesie: Xensze, Srem, Brodnicza, Borovo, Gluchovo, et alie omnes que consistunt ultra fluvium qui dicitur Obra usque Padlamgoram et Visoczko, ita ut Vartam non excedat, censuimus limitandum. Minorem vero archidiaconatum, prout hec ecclesie sunt site: Vronki, Othorovo, Vilczina, Czeradz, Buk, Ptaskovo, Lanki, et omnes alie inter Vartam et Padlamgoram et usque ad Vratislaviensem et Lubucensem dyoceses, limitamus. Damus autem archidiaconis, maiori, medio et minori, per nos canonice institutis et per nostros successores in posterum instituendis, liberam et generalem archidiaconatus sui administracionem, ita ut singulus eorum in suo archidiaconio habeat plenariam auctoritatem visitandi, corrigendi, excommunicandi, absolvendi, suspendendi, interdicendi, relaxandi, iudicandi, et alias penas spiritales secundum canones et sue discrecionis moderamen delinquentibus infligendi, quibus animarum remedium procuratur. Et si que pene pecuniarie delinquentibus forsitan fuerint infligende, has archidiaconi non valeant imponere vel eciam exigere nisi episcopo requisito, quas episcopus, si sibi videbitur, moderabit secundum graciam a Deo sibi datam. Quod si archidiaconi huic nostro decreto contraire ausi fuerint, aut eciam pro visitacione pecuniam vel solucionem aliquam pro penis acceperint vel extorserint, ab exsecucione archidiaconalis officii ipsos suspendimus ipso facto, donec pecuniam quam receperint, in presencia episcopi eis a quibus receperint restituant, et per episcopum exsecucioni officii restaurentur. Damus eciam eis exercicium omnium iurium universalium et singularium, que archidiaconis de iure vel consuetudine competunt vel possunt competere in nostro episcopio in futurum, cuilibet eorum stallum in choro et vocem in Capitulo, maiori stallum in sinistro choro post decanum, medio in dextero choro post canthorem, minori in sinistro choro post custodem. Maiori pro tytulo ecclesiam de Gczecz cum eiusdem ecclesie redditibus assingnantes, temporibus et locis congruentibus assingnaturi tytulos medio et minori. Et hec omnia salvo iure Sedis apostolice statuimus et eciam ordinamus. Verum, ne illa que nostris aguntur temporibus, mutabilitas pertransiens aut vetustas deterat, neve ex hoc calumpniis in posterum materia subministretur, litteram hanc puplicam (sic) conscribi et sigillo nostro nostrique Capituli fecimus communiri. Actum et datum in Capitulo Poznaniensi II Kalendas Februarii, anno gracie Domini MCC nonogesimo octavo, Indiccione undecima, presentibus dominis fratribus et canonicis nostris: Iohanne preposito, magistro Gregorio decano, Henrico scolastico, Thomca custode, Iacobo nostro canonico et cancellario, magistro Paulo, Friczkone, Iacobo, Henrico, Vlostone, Michaele, Salomone, Predvoyo, Nycolao, Borislao, Sdislao, Heyncza, Gercone, Fredrico, Petro, Stephano, et aliis multis Capituli nostri Poznaniensis canonicis fidedingnis.'

In [35]:
x = remove_characters(lettre,characters_to_remove).lower().lstrip().rstrip().replace('j','i')

In [41]:
y

'te redargutionis et culpae non uidimus nota carentem dum contra et  qui homicidium sub umbra nostri nominis commisisse dicuntur tam lente tam segniter audiuimus processisse sequitur etenim quod dum maleficiorum excessus huiusmodi iusto presidis rigore non stringitur uitiosa sibi iam penae remissio nutritiua sit criminum et facinorosis ex hoc crescat audacia cum liber quodam modo transitus uiolentiae prebeatur'

In [4]:
x = viard_lemmes

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


lemmas = []
for i in tqdm(range(len(x))):
    sentences: List[str] = [x[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        lemmas.append(dic)

fatal: HEAD n'est pas un nom d'objet valide
100%|███████████████████████████████████████████| 65/65 [02:26<00:00,  2.25s/it]


In [5]:
liste_pos = ['ADJqua','VER','ADV','NOMcom']
maxime = []
for i in range(len(lemmas)):
    alexandra = []
    for j in lemmas[i]:
        alexandra.append(j.get('pos'))
    maxime.append(alexandra)

franck = []
for i in range(len(lemmas)):
    clemi = []
    for j in lemmas[i]:
        clemi.append(j.get('lemma'))
    franck.append(clemi)

In [6]:
lemmes_viard = copies(franck,list(dico_un.values()))
zoe['lemmes_viard'] = lemmes_viard
pos_viard = copies(maxime,list(dico_un.values()))
zoe['pos_viard'] = pos_viard

In [7]:
x = df['compare'].tolist()

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


lemmes_pdv = []
for i in tqdm(range(len(x))):
    sentences: List[str] = [x[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        lemmes_pdv.append(dic)

100%|█████████████████████████████████████████| 363/363 [06:35<00:00,  1.09s/it]


In [8]:
liste_pos = ['ADJqua','VER','ADV','NOMcom']
thibault = []
for i in range(len(lemmes_pdv)):
    aline = []
    for j in lemmes_pdv[i]:
        aline.append(j.get('pos'))
    thibault.append(aline)
romain = []
for i in range(len(lemmes_pdv)):
    dorothee = []
    for j in lemmes_pdv[i]:
        dorothee.append(j.get('lemma'))
    romain.append(dorothee)

In [9]:
lemmes_pdv = romain * len(viard)
zoe['lemmes_pdv'] = lemmes_pdv
pos_pdv = thibault * len(viard)
zoe['pos_pdv'] = pos_pdv

In [10]:
zoe['lemmes_viard'] = zoe['lemmes_viard'].apply(join)
zoe['lemmes_pdv'] = zoe['lemmes_pdv'].apply(join)
zoe['pos_pdv'] = zoe['pos_pdv'].apply(join)
zoe['pos_viard'] = zoe['pos_viard'].apply(join)

In [11]:
zoe.to_excel('zoe.xlsx')

In [23]:
df= pd.read_excel('zoe.xlsx')

In [34]:
from nltk.util import skipgrams
from nltk.tokenize import word_tokenize

In [35]:
def skip_grams(string):
    gram_length = 3
    skip = 1
    liste = string.split()
    final = list(skipgrams(liste, gram_length, skip))
    
    return final

In [ ]:
df['skip_viard'] = df['viard'].progress_apply(skip_grams)
df['skip_pdv'] = df['pdv'].progress_apply(skip_grams)

 13%|████▊                                | 3093/23595 [00:15<01:11, 285.95it/s]

3637

In [ ]:
un = copies(maxime,thibault)
deux = thibault*len(maxime)
cinq = copies(franck,thibault)
six = romain*len(maxime)
zoe['viard_cos'] = un
zoe['viard_cos'] = zoe['viard_cos'].apply(join)

zoe['pdv_cos'] = deux
zoe['pdv_cos'] = zoe['pdv_cos'].apply(join)
zoe['lemmes_viard'] = cinq
zoe['lemmes_viard'] = zoe['lemmes_viard'].apply(join)

zoe['lemmes_pdv'] = six
zoe['lemmes_pdv'] =zoe['lemmes_pdv'].apply(join)
zoe['cosine'] = zoe.progress_apply(lambda x:cosine_sim(x['viard_cos'],x['pdv_cos']),axis=1)

In [185]:
#création du dictionnaire de pos

dic1 = [item for sublist in lemmes_pdv for item in sublist]
dic2 = [item for sublist in lemmas for item in sublist]

dic1.extend(dic2)

mots = []
part_of_speech = []

for dic in dic1:
    mots.append(dic.get('form'))
    part_of_speech.append(dic.get('pos'))

dic3 = dict(zip(mots,part_of_speech))

In [ ]:
mathieu = loic[loic['same'] == False].reset_index(drop=True)
mathieu['deux'] = mathieu['delete'].apply(unique_tri_dans_quadri)
mathieu = mathieu[mathieu['deux'] != False].reset_index(drop=True)

In [ ]:
mathieu['delete'] = mathieu['delete'].apply(sans_doublons)
mathieu['len'] = mathieu['delete'].apply(length)
mathieu = mathieu[mathieu['len'] > 1].reset_index(drop=True)
mathieu = mathieu[['indice_viard','viard','lemmes_viard','indice_pdv','pdv','lemmes_pdv','delete']]

In [ ]:
mathieu['pos'] = mathieu['delete'].apply(pos_list)

In [ ]:
mathieu['quadri_viard'] = mathieu['viard'].progress_apply(split).progress_apply(string_to_quadrigrams)
mathieu['quadri_pdv'] = mathieu['pdv'].progress_apply(split).progress_apply(string_to_quadrigrams)

In [112]:
zoe

NameError: name 'zoe' is not defined

In [ ]:
x = mathieu['delete'][0]

In [ ]:
pos_list(x)

NameError: name 'zoe' is not defined

In [ ]:
un = copies(maxime,thibault)
deux = thibault*len(maxime)
trois = copies(viard_keys,thibault)
quatre = pdv_keys*len(maxime)
cinq = copies(franck,thibault)
six = romain*len(maxime)
zoe['viard_cos'] = un
zoe['viard_cos'] = zoe['viard_cos'].apply(join)

zoe['pdv_cos'] = deux
zoe['pdv_cos'] = zoe['pdv_cos'].apply(join)
zoe['lemmes_viard'] = cinq
zoe['lemmes_viard'] = zoe['lemmes_viard'].apply(join)

zoe['lemmes_pdv'] = six
zoe['lemmes_pdv'] =zoe['lemmes_pdv'].apply(join)
zoe['cosine'] = zoe.progress_apply(lambda x:cosine_sim(x['viard_cos'],x['pdv_cos']),axis=1)
zoe = zoe[['indice_viard','viard','indice_pdv','pdv','lemmes_viard','lemmes_pdv','cosine']]
zoe = zoe[zoe['cosine'] > 0.05]

In [ ]:
test['quadri_viard'] = test['viard'].progress_apply(split).apply(string_to_quadrigrams)

In [ ]:
zoe['trigram'] = zoe.progress_apply(lambda x: avoir_trigram_commun(x['viard'],x['pdv']),axis=1)

In [ ]:
test = zoe[zoe['trigram'] == True]

In [ ]:
anais = test[test['cosine'] > 0.05].reset_index(drop=True)

In [ ]:
han = []
trigrams = []

for i in tqdm(range(len(anais))):
    inter = []
    for tri in string_to_trigrams(anais['viard'][i].split()):
        if tri in string_to_trigrams(anais['pdv'][i].split()):
            inter.append(tri)
    han.append(len(inter))
    trigrams.append(inter)

In [ ]:
anais['trigrams'] = trigrams
anais['len'] = han

In [ ]:
trigrams_delete = [['tam', 'presentibus', 'quam'],['notum', 'facimus', 'uniuersis'],
                  ['presentibus', 'quam', 'futuris'],['uniuersis', 'tam', 'presentibus'],
                  ['quam', 'futuris', 'quod'],['de', 'speciali', 'gratia'],
                  ['et', 'ex', 'certa'],['ex', 'certa', 'scientia'],['speciali', 'gratia', 'et'],
                  ['futuris', 'quod', 'nos'],['presentes', 'litteras', 'inspecturis'],
                   ['uniuersis', 'presentes', 'litteras'],
                  ['facimus','uniuersis','tam'],['de','regno','nostro'],
                  ['in','regno','nostro'],['et','regni','nostri'],['regni','nostri','et'],
                   ['presentibus', 'et', 'futuris'],['per','regnum','nostrum']]

In [ ]:
anais['grams'] = anais['trigrams'].apply(remove_lists)
anais['empty'] = anais['grams'].apply(empty_lists)
lucie = anais[anais['empty'] == False].reset_index(drop=True)
lucie = lucie[['indice_viard','viard','indice_pdv','pdv','lemmes_viard','lemmes_pdv','grams']]

In [ ]:
lucie['len'] = lucie['grams'].apply(length)
michelle = lucie[lucie['len']>1]

In [ ]:
michelle['same'] = michelle['grams'].apply(same)

In [ ]:
emile = michelle[michelle['same'] == False].reset_index(drop=True)

In [ ]:
#création du dictionnaire de pos

dic1 = [item for sublist in lemmes_pdv for item in sublist]
dic2 = [item for sublist in lemmas for item in sublist]

dic1.extend(dic2)

mots = []
part_of_speech = []

for dic in dic1:
    mots.append(dic.get('form'))
    part_of_speech.append(dic.get('pos'))

dic3 = dict(zip(mots,part_of_speech))

In [ ]:
emile = emile[['indice_viard','viard','indice_pdv','pdv','lemmes_viard','lemmes_pdv','grams']]

In [ ]:
emile['pos'] = emile['grams'].apply(pos_list)

In [ ]:
emile['empty'] = emile['pos'].apply(empty_lists)

In [ ]:
vincent = emile[emile['empty'] == False].reset_index(drop=True)

In [ ]:
vincent = vincent[['indice_viard','viard','indice_pdv','pdv','lemmes_viard','lemmes_pdv','grams','pos']]

In [ ]:
chahan = vincent.progress_apply(lambda x: bold_texts(x['viard'],x['pdv']),axis=1)

In [ ]:
vincent['resultat'] = chahan

In [85]:
final_texte = ' '.join(x)
pierre_final = ' '.join(y)

print('----------')
display(Markdown(final_texte))
print('----------')
display(Markdown(pierre_final))    
print('----------')

----------


sane cum de culminis nostri fulgore **uelut** **ex** **sole** **radii** **sic** omnium largiente conditore ******prodeant****** ******dignitates****** **ut** ****prime**** **lucis** **integritas** **minorati** **luminis** **non** ********sensiat******** **detrimenta** nam tanto **magis** regale ****sceptrum**** ****extollitur**** tantoque ********curia******** ****regiminis**** ****sollicitudinibus**** **releuatur** **quanto** **tribunal** **ipsius** **digniores** **circumspicit** **contribules** **hac** **igitur** **consideratione** ducti quia regni francie **moderamur** **habenas** **solii** **nostri** **decus** **tam** **ueterum** **dignitatum** **ornatibus** confoueri **quam** etiam nouis ardemus honoribus ampliari et licet ad quorumlibet nostrorum fidelium munifice prosequenda seruicia liberalitas nostra quadam generalitate sit procliuis illis tamen est quadam specialitate debitrix et eorum uota prosequi liberaliter affectamus in quibus sanguinis connexitatem et uinculum agnoscimus et puram fidei constanciam inuenimus ut ipsi feruore deuocionis accensi reddantur de bono in melius circa maiestatem regiam et eius obsequia promptiores et ne percepta dudum obsequia que conscribi perhenniter in libro memorie meruerunt preminentia gradus aboleat uel incompensata relinquat

----------


de fulgore throni cesarei **uelut** **ex** **sole** **radii** **sic** certe ****prodeunt**** ****dignitates**** **ut** ****prime**** **lucis** **integritas** **non** ********sentiat******** **minorati** **luminis** **detrimenta** tantoque **magis** imperiale ****sceptrum**** ****extollitur**** tanto ********cura******** ****regiminis**** pluria ****sollicitudinibus**** **releuatur** **quanto** **tribunal** **ipsius** **digniores** in circuitu **circumspicit** **contribules** regiones **hac** **igitur** **consideratione** commoti qui celesti prouidentia romani imperii **moderamur** **habenas** **solii** **nostri** **decus** **tam** **ueterum** **dignitatum** **ornatibus** confouemus **quam** nouis honoribus ampliamur

----------
